RAG - REACT - TOOL

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) 

True

In [2]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
from langchain.chains import LLMMathChain, LLMChain

# Create tools for the agent
################### first custom tool #############

#create the WMI serach tool
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

#Load the documents
from langchain_community.document_loaders import DirectoryLoader
#loader = TextLoader("./bella_vista.txt")
DATA_PATH = "mydata"
loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
docs = loader.load()


#Split the documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
documents = text_splitter.split_documents(docs)

#for doc in documents:
    #print("\n")
    #print(doc)

print("####### FIRST 10 docs ############")
for i in range(10):
    print("\n")
    print(documents[i])

print("####### LAST 10 docs ############")
x = documents[::-1]
i = 0
while (i < 10):
    print("\n")
    print(x[i])
    i+= 1

print("############## # of Documents ##################")
print(len(documents))

#Initiatlise the Embeddings
embeddings = OpenAIEmbeddings()

#Initialize FAISS vector store
from langchain.vectorstores.faiss import FAISS
vectorstore = FAISS.from_documents(documents, embeddings)
#Save the data
vectorstore.save_local("index") # newer FAISS versions can not be serialized with pickle

#Load the vectoestore in memory to be used later
vectorstore = FAISS.load_local("index", embeddings, allow_dangerous_deserialization=True)

from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class WMSearchTool(BaseTool):
    name = "WM search"
    description = "useful when you need to answer questions about WM product"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

################## custom tool end ####################



####### FIRST 10 docs ############


page_content='PUSH POSSIBLETM\n\nWAREHOUSE MANAGEMENT AND TRANSPORT EXECUTION FOR IBM i\n\nRELEASE NOTES 2024\n\n0' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='0\n\nWMi AND TEi RELEASE NOTES\n\nCopyright and Disclaimer' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='Copyright © 2024 Manhattan Associates, Inc. All rights reserved.' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='This documentation, as well as the software described in it, is furnished under license and may be' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='license and may be used or copied only in accordance with the terms of such license. The' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='such license. The information in this documentation is furnished for informational use only, is' metadata={'source': 'mydata/WMI-RN.pdf'}


page_content='use only, is subject to change without notice, and should not be construed as a commitment b

In [3]:
from langchain_openai import OpenAI
llm=OpenAI()

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

#Another math tool as REPL not working as expected
problem_chain = LLMMathChain.from_llm(llm=llm)
math_tool = Tool.from_function(name="Calculator",
                func=problem_chain.run,
                description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.")



In [8]:
#Add Memory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

In [5]:

# Initialize the ChatOpenAI model (gpt-4-turbo-preview) with a temperature of 0. Utilize gpt-3.5-turbo if you use the free plan
llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)
#llm=OpenAI()

#Answer the following questions in GERMAN as best you can.
# Define a template for answering questions
template = '''
Answer the following questions as best you can.
Questions: {q}
'''
# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template)

# Pull the react prompt from the hub
prompt = hub.pull('hwchase17/react')


# Combine the tools into a list
tools = [WMSearchTool(), math_tool, python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    memory=memory,
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)


In [9]:
#question = 'Does WMi support json interfaces?'
#question = 'Who is the current prime minister of the UK?'
#question = 'what is the answer to 3 ** 3.9'
#question = 'Tell me about Napoleon Bonaparte early life'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active Supply Chain'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active® Warehouse Management'
#question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active Supply Chain, enabled by default?'
question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active® Warehouse Management, enabled by default?'

output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})

question = 'Is the feature enabled by default?'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})




> Entering new AgentExecutor chain...
To find out if the feature "Creation of Multiple Process Needs for an Item" is enabled by default in Manhattan Active® Warehouse Management, I should look up the official documentation or any reliable source of information regarding the software's features and settings.

Action: WM search

Action Input: "Creation of Multiple Process Needs for an Item" in Manhattan Active Warehouse Management enabled by default


Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID 82ef2f9a-53e1-4039-b22f-2697e7a1f05c, but expected {'tool'} run.")


Creation of Multiple Process Needs for an Item
Creation of Multiple Process Needs for an Item
The solution allows users to create multiple process needs for an item, allowing them to add more
To achieve this functionality, enable the “Should the system allow multiple Process needs creationThe information provided does not explicitly state whether the feature "Creation of Multiple Process Needs for an Item" is enabled by default in Manhattan Active® Warehouse Management. It mentions that to achieve this functionality, one needs to enable a specific setting, but it does not clarify if this setting is enabled by default.

Final Answer: The documentation does not explicitly state whether the feature "Creation of Multiple Process Needs for an Item" is enabled by default in Manhattan Active® Warehouse Management. It suggests that a specific setting needs to be enabled to use this functionality, implying it may not be enabled by default.

> Finished chain.


> Entering new AgentExecutor chain